## Семинар 5: "Улучшение сходимости нейросетей"

ФИО: Усцов Артем Алексеевич

In [ ]:
from train_utils import train, compare_results

import numpy as np
from sklearn.model_selection import train_test_split

import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms

import matplotlib.pyplot as plt
%matplotlib inline

На этом семинаре мы попробуем улучшить результаты, полученные на предыдущем занятии
Для этого нам понадобятся следующие вещи:
* Dropout
* Batch Normalization
* Инициализация весов

### Часть 1: Инициализация весов

На лекции доказывалось, что при инициализации He и Glorot дисперсия активаций градиентов в каждом слое будут примерно равны. Давайте проверим это. 

In [ ]:
# Dataloader
to_numpy = lambda x: x.numpy()
transform = transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                    ])
train_dataset = MNIST('.', train=True, download=True, transform=transform)
test_dataset = MNIST('.', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=True)

In [ ]:
images_train, labels_train = next(iter(train_loader))

In [ ]:
## Usage example:
for X, y in train_loader:
    X = X.view(X.size(0), -1)
    X = X.numpy() ### Converts torch.Tensor to numpy array
    y = y.numpy()
    pass

In [ ]:
plt.figure(figsize=(6, 7))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.imshow(X[i].reshape(28, 28), cmap=plt.cm.Greys_r)
    plt.title(y[i])
    plt.axis('off')

<i> 1.1 </i> Инициализируйте полносвязную сеть нормальным шумом N(0, 0.1) с архитектурой 784 -> 500 x (10 раз) -> 10. В качестве активации возьмите tanh

In [ ]:
NUM_EPOCHS = 20

In [ ]:
def init_layer(layer, mean=0, std=1):
    # Тут надо быть аккуратным — можно случайно создать копию и менять значения у копии
    weight = layer.state_dict()['weight']
    bias = layer.state_dict()['bias']
    bias.zero_()
    #1 - веса сети
    #2 - нули ...
    #bias = torch.zeros_like(bias)
    weight.normal_(mean=0, std=std)

def forward_hook(self, input_, output):
    std = input_[0].std().item()
    print('forward', std)

def backward_hook(self, grad_input, grad_output):
    std = grad_input[0].std().item()
    print('backward', std)

    
# пример:
layer = nn.Linear(28*28, 10)
layer.register_forward_hook(forward_hook)
layer.register_backward_hook(backward_hook)

# сюда надо подставить другие параметры
init_layer(layer, 0.0, 0.1)

In [ ]:
sizes = [784] + [500] * 10 + [10]
layers = []

def normal(size_input, size_output):
    return 0.1
    #return 1

def xavier(size_input, size_output):
    d = 2 / (size_input + size_output)
    return np.sqrt(d)

def good_grad(size_input, size_output):
    d = 1 / size_output
    return np.sqrt(d)

#init_func = normal
init_func = xavier
#init_func = good_grad

for size_input, size_output in zip(sizes, sizes[1:]):
    
    layer = nn.Linear(size_input, size_output)
    layer.register_forward_hook(forward_hook)
    layer.register_backward_hook(backward_hook)
    init_layer(layer, 0.0, init_func(size_input, size_output)) # сюда надо подставить другие параметры
    
    layers.append(layer)
    #layers.append(nn.Tanh())
    #layers.append(nn.Sigmoid())
    layers.append(nn.Tanh())
    
print(len(layers))
del layers[-1]

<i>1.2 Пропустите батч изображений через нейронную сеть и вычислите дисперсию активаций. Затем вычислите градиент и получите дисперсию градиентов. Сравните эти значения между собой для разных слоев.</i>

In [ ]:
network = nn.Sequential(*layers)

#пример:
n_objects = 100
X = images_train[:n_objects].view(n_objects, -1).data
y = labels_train[:n_objects].data
activations = network(X)
loss_fn = torch.nn.NLLLoss()
optimizer = torch.optim.Adam(network.parameters(), lr=0.001) 
loss = loss_fn(activations, y)
loss.backward()

<i>1.3 Повторите эксперимент для инициализаций He и Xavier (формулы есть в лекции).</i>

### Xavier-initialisation

In [ ]:
##### YOUR CODE HERE #####
def xavier_init_layer(layer, a_range):
    weight = layer.state_dict()['weight']
    bias = layer.state_dict()['bias']
    bias.zero_()
    weight.uniform_(-a_range, a_range)
    
def xavier(size_input, size_output):
    d = 6 / (size_input + size_output)
    return np.sqrt(d)

In [ ]:
sizes = [28*28, 64, 32] + [16] * 30 + [10]
layers = []

for size_input, size_output in zip(sizes, sizes[1:]):
    
    layer = nn.Linear(size_input, size_output)
    layer.register_forward_hook(forward_hook)
    layer.register_backward_hook(backward_hook)

    xavier_init_layer(layer, xavier(size_input, size_output))
    
    layers.append(layer)
    layers.append(nn.Tanh())
    #layers.append(nn.Sigmoid())
    #layers.append(nn.ReLU())
    
print(len(layers))
del layers[-1]

In [ ]:
network = nn.Sequential(*layers)

n_objects = 100
X = images_train[:n_objects].view(n_objects, -1).data
y = labels_train[:n_objects].data

activations = network(X)

loss_fn = torch.nn.NLLLoss()
optimizer = torch.optim.Adam(network.parameters(), lr=0.001)
loss = loss_fn(activations, y)
loss.backward()

### He-initialisation

In [ ]:
def he_init_layer(layer, mean=0, std=1):
    weight = layer.state_dict()['weight']
    bias = layer.state_dict()['bias']
    bias.zero_()
    weight.normal_(mean=mean, std=std)
    
def he_forward(size_input, size_output):
    return np.sqrt(2 / size_input)

def he_backward(size_input, size_output):
    return np.sqrt(2 / size_output)

In [ ]:
sizes = [28*28, 64, 32] + [16] * 30 + [10]
layers = []

for size_input, size_output in zip(sizes, sizes[1:]):
    
    layer = nn.Linear(size_input, size_output)
    layer.register_forward_hook(forward_hook)
    layer.register_backward_hook(backward_hook)

    #he_init_layer(layer, 0, he_forward(size_input, size_output))
    he_init_layer(layer, 0, he_backward(size_input, size_output))
    
    layers.append(layer)
    #layers.append(nn.Tanh())
    #layers.append(nn.Sigmoid())
    layers.append(nn.ReLU())
    
print(len(layers))
del layers[-1]

In [ ]:
network = nn.Sequential(*layers)

n_objects = 100
X = images_train[:n_objects].view(n_objects, -1).data
y = labels_train[:n_objects].data

activations = network(X)

loss_fn = torch.nn.NLLLoss()
optimizer = torch.optim.Adam(network.parameters(), lr=0.001)
loss = loss_fn(activations, y)
loss.backward()

<i> 1.4 Сделайте выводы по первой части </i>

Для функции активации Tanh лучше использовать инициализацию Xavier(во избежание стремительных взрывов и затуханий градиентов), для функции активации ReLU лучше использовать инициализацию весов He.


### Часть 2: Dropout

Другим полезным слоем является __Dropout.__ В нем с вероятностью 1-p зануляется выход каждого нейрона. Этот слой уже реализован в pyTorch, поэтому вновь реализовывать его не интересно.  
Давайте реализуем __DropConnect__ — аналог Dropout. В нем с вероятностью 1-p зануляется каждый вес слоя.

In [ ]:
class TestNetwork(nn.Module):
    def __init__(self, final_part):
        super().__init__()    
        
        channels = 1
        
        self.conv_layers = nn.Sequential(
            nn.Conv2d(channels, 2, 3, padding=1),    
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Conv2d(2, 4, 3, padding=1),            
            nn.MaxPool2d(2),
            nn.ReLU(),
        )
        
        #input_size = 7 * 7 * 4 = 196
        self.flatten = nn.Flatten()
        self.final_part = final_part
        self.log_softmax = nn.LogSoftmax(1)        
        
    def forward(self, x):
        x = self.conv_layers(x)
        x = self.flatten(x)
        x = self.final_part(x)
        return self.log_softmax(x)

<i> 2.1 Реализуйте линейный слой с DropConnect </i>

In [ ]:
# полезная функция: .bernoulli_(p)
# не забывайте делать requires_grad=False у маски
# помните, что в вычислениях должны участвовать Variable, а не тензоры


class DropConnect(nn.Module):
    def __init__(self, input_dim, output_dim, p=0.5):
        super(DropConnect, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        self.p = p

    def forward(self, x):    
        mask = torch.zeros_like(self.linear.weight) + self.p
        if self.training:          
            mask.bernoulli_(self.p)

        mask = mask.data
        output = F.linear(x, self.linear.weight * mask, self.linear.bias)
        return output

<i> 
2.2 Сравните графики обучения нейроных сетей:  
    
1. Свертки из TestNetwork -> 128 -> 128 -> 10 с ReLU и Dropout между всеми слоями;  
    
2. Свертки из TestNetwork -> 128 -> 128 -> 10 с ReLU и DropConnect вместо всех линейных слоев;  
    
</i>

### Baseline (классическая полносвязная)

In [ ]:
sizes = [196, 128, 128, 10]
layers = []
for size_input, size_output in zip(sizes, sizes[1:]):
    layers.append(nn.Linear(size_input, size_output))
    layers.append(nn.ReLU())

# исключим активационную на последнем слое
del layers[-1]
print()
[print(f'{i}: {layer}') for i, layer in enumerate(layers)]

In [ ]:
%%time

network = TestNetwork(nn.Sequential(*layers))
tr, ts, tr_ac, ts_ac = train(network, train_loader, test_loader, NUM_EPOCHS, 0.001)

### Dropout и ReLU после каждого слоя

In [ ]:
sizes = [196, 128, 128, 10]
w_dropout_layers = []
for size_input, size_output in zip(sizes, sizes[1:]):
    w_dropout_layers.append(nn.Linear(size_input, size_output))

    # вероятность зануления нейрона в слое=0.3
    w_dropout_layers.append(nn.Dropout(0.3))
    w_dropout_layers.append(nn.ReLU())

# исключим dropout и relu с последних слоев
del w_dropout_layers[-2:]
print()
[print(f'{i}: {layer}') for i, layer in enumerate(w_dropout_layers)]

In [ ]:
%%time

network = TestNetwork(nn.Sequential(*w_dropout_layers))
tr_1, ts_1, tr_ac_1, ts_ac_1 = train(network, train_loader, test_loader, NUM_EPOCHS, 0.001)

### ReLU и DropConnect вместо линейных слоев

In [ ]:
sizes = [196, 128, 128, 10]
dropconnect_layers = [nn.ReLU(), DropConnect(196, 10, p=0.8)]
# dropconnect_layers = []
# for size_input, size_output in zip(sizes, sizes[1:]):
# #     layer = nn.Linear(size_input, size_output)
#     dropconnect_layers.append(DropConnect(size_input, size_output, 0.8))
#     dropconnect_layers.append(nn.ReLU())

# del dropconnect_layers[-1]
print()
[print(f'{i}: {layer}') for i, layer in enumerate(dropconnect_layers)]

In [ ]:
%%time

network = TestNetwork(nn.Sequential(*dropconnect_layers))
tr_2, ts_2, tr_ac_2, ts_ac_2 = train(network, train_loader, test_loader, NUM_EPOCHS, 0.001)

В test-time стохастичность Dropout убирают и заменяют все веса на их ожидаемое значение: $\mathbb{E}w = pw + (1-p)0 = pw$.

<i> 2.3 Сделайте выводы по второй части. </i>

Качество с Dropout лучше, чем с DropConnect.
При этом отчетливо видно, что качество модели на трейне много хуже, чем на тесте. Стратегия зануления нейронов или слоев "затрудняет" обучение модели.

### Часть 3: Batch Normalization

Наконец, давайте рассмотрим Batch Normalization. Этот слой вычитает среднее и делит на стандартное отклонение. Среднее и дисперсия вычисляются по батчу независимо для каждого нейрона. У этого слоя есть две важные проблемы: его нельзя использовать при обучении с размером батча 1 и он делает элементы батча зависимыми. Давайте реализуем аналог батч нормализации: <a href=https://arxiv.org/pdf/1607.06450.pdf>Layer normalization</a>. В layer normalization среднее и дисперсия вычисляются по активациям нейронов, независимо для каждого объекта.

<i> 3.1 Реализуйте Layer Normalization </i>

In [ ]:
# полезные функции: .std(dim), .mean(dim)

class LayerNormalization(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.alpha = nn.Parameter(torch.ones(input_dim))
        self.beta = nn.Parameter(torch.zeros(input_dim))
        
    def forward(self, x):
        output = self.alpha * (x - x.mean()) / (x.std() + 1e-8) +  self.beta
        return output

<i> 
3.2 Сравните графики обучения нейроных сетей:  
    
1. Свертки из TestNetwork -> 128 -> 128 -> 10 с ReLU и Batch normalization между всеми слоями  
    
2. Свертки из TestNetwork -> 128 -> 128 -> 10 с ReLU и Layer normalization между всеми слоями  
    
</i>

### ReLU и Batch normalization

In [ ]:
##### YOUR CODE HERE #####
sizes = [196, 512, 128, 10]
relu_batch_normed_layers = []
for in_dim, out_dim in zip(sizes, sizes[1:]): 
    relu_batch_normed_layers.append(nn.Linear(in_dim, out_dim))
    relu_batch_normed_layers.append(nn.BatchNorm1d(out_dim))
    relu_batch_normed_layers.append(nn.ReLU())

del relu_batch_normed_layers[-2:]
print()
[print(f'{i}: {layer}') for i, layer in enumerate(relu_batch_normed_layers)]

In [ ]:
%%time
network = TestNetwork(nn.Sequential(*relu_batch_normed_layers))
tr_4, ts_4, tr_ac_4, ts_ac_4 = train(network, train_loader, test_loader, NUM_EPOCHS, 0.001)

### ReLU и Layer normalization

In [ ]:
sizes = [196, 128, 128, 10]
relu_layer_normed_layers = []
for in_dim, out_dim in zip(sizes, sizes[1:]): 
    relu_layer_normed_layers.append(nn.Linear(in_dim, out_dim))
    relu_layer_normed_layers.append(LayerNormalization(out_dim))
    relu_layer_normed_layers.append(nn.ReLU())

del relu_layer_normed_layers[-2:]
print()
[print(f'{i}: {layer}') for i, layer in enumerate(relu_layer_normed_layers)]

In [ ]:
%%time
network = TestNetwork(nn.Sequential(*relu_layer_normed_layers))
tr_5, ts_5, tr_ac_5, ts_ac_5 = train(network, train_loader, test_loader, NUM_EPOCHS, 0.001)

### Часть 4: Сравнение всех методов

In [ ]:
compare_results(loss_results=[ts, ts_1, ts_2, ts_4, ts_5], 
            acc_results=[ts_ac, ts_ac_1, ts_ac_2, ts_ac_4, ts_ac_5],
            labels=["Baseline", "Dropout", "Dropconnect",
                    "BatchNormed", "LayerNormed"],
            )

<i> 3.3 Сделайте выводы по третьей части </i>

Batch и Layer не несут в себе каких-либо особо видимых различий с точки зрения качества или характера поведения обучения модели, однако Batch оказывается чуть лучше.

На графике сравнений, стоит отметить, что методы с нормализацией очень быстро обучаются и выходят на линию "насыщения" уже после 1-2 эпохи  
А вот, Dropconnect, наоборот ведет себя даже хуже, чем эталанная модель.
Возможно, стоит поиграться с вероятностью зануления весов, что так же относится и к подходу Dropout